# Session #4 Homework

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

Downloading data

In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv

--2021-10-05 12:46:22--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182489 (178K) [text/plain]
Saving to: ‘CreditScoring.csv.5’

CreditScoring.csv.5 100%[===================>] 178.21K  --.-KB/s    in 0.003s  

2021-10-05 12:46:22 (61.0 MB/s) - ‘CreditScoring.csv.5’ saved [182489/182489]



In [3]:
df = pd.read_csv("CreditScoring.csv")
df.head()

,Status,Seniority,Home,Time,Age,Marital,Records,Job,Expenses,Income,Assets,Debt,Amount,Price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


### Preparation

 **We'll talk about this dataset in more details in week 6. But for now, use the following code to get started



In [4]:
df.columns = df.columns.str.lower()

Some of the features are encoded as numbers. Use the following code to de-code them:



In [5]:
status_values = {
    1: 'ok',
    2: 'default',
    0: 'unk'
}

df.status = df.status.map(status_values)


home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}

df.home = df.home.map(home_values)

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

df.marital = df.marital.map(marital_values)

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

df.records = df.records.map(records_values)

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

df.job = df.job.map(job_values)


Prepare the numerical variables:

In [6]:
for c in ['income', 'assets', 'debt']:
    df[c] = df[c].replace(to_replace=99999999, value=0)

Remove clients with unknown default status

In [7]:
df = df[df.status != 'unk'].reset_index(drop=True)


Create the target variable



In [8]:
df['default'] = (df.status == 'default').astype(int)
del df['status']


In [9]:
df

,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price,default
0,9,rent,60,30,married,no,freelance,73,129,0,0,800,846,0
1,17,rent,60,58,widow,no,fixed,48,131,0,0,1000,1658,0
2,10,owner,36,46,married,yes,freelance,90,200,3000,0,2000,2985,1
3,0,rent,60,24,single,no,fixed,63,182,2500,0,900,1325,0
4,0,rent,36,26,single,no,fixed,46,107,0,0,310,910,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4449,1,rent,60,39,married,no,fixed,69,92,0,0,900,1020,1
4450,22,owner,60,46,married,no,fixed,60,75,3000,600,950,1263,0
4451,0,owner,24,37,married,no,partime,60,90,3500,0,500,963,1
4452,0,rent,48,23,single,no,freelance,49,140,0,0,550,550,0


In [10]:
df.dtypes

seniority     int64
home         object
time          int64
age           int64
marital      object
records      object
job          object
expenses      int64
income        int64
assets        int64
debt          int64
amount        int64
price         int64
default       int64
dtype: object

In [11]:
categorical = ["home","marital","records","job"]
numerical = ["seniority","time","age","expenses","income","assets","debt","price"]

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
df_full_train, df_test = train_test_split(df, test_size = 0.2 , random_state = 1)

df_train , df_val = train_test_split(df_full_train, test_size =0.25, random_state = 11)

y_train = df_train.reset_index(drop = True)
y_val = df_val.reset_index(drop = True)
y_test = df_test.reset_index(drop = True)

y_train = df_train.default.values
y_val = df_val.default.values
y_test = df_test.default.values

del df_train["default"]
del df_val["default"]
del df_test["default"]


In [14]:
len(df_train),len(df_val),len(df_test)

(2672, 891, 891)

### Question 1

ROC AUC could also be used to evaluate feature importance of numerical variables.

Let's do that

For each numerical variable, use it as score and compute AUC with the "default" variable
Use the training dataset for that
If your AUC is < 0.5, invert this variable by putting "-" in front

(e.g. -df_train['expenses'])

AUC can go below 0.5 if the variable is negatively correlated with the target varialble. You can change the direction of the correlation by negating this variable - then negative correlation becomes positive.

Which numerical variable (among the following 4) has the highest AUC?

seniority / time / income / debt

In [15]:
from sklearn.metrics import roc_auc_score

In [16]:
for c in numerical:
    auc = roc_auc_score(y_train,df_train[c])
    if auc < 0.5:
        auc = roc_auc_score(y_train,-df_train[c])
    print(c , auc)

seniority 0.7069246333952217
time 0.5623193380546322
age 0.5701048256195315
expenses 0.503575716811011
income 0.6852917551446963
assets 0.6365498571380924
debt 0.5009238079091021
price 0.5057443550090609


In [17]:
## **seniority 0.7069246333952217**

### Question 2

 What's the AUC of this model on the validation dataset? (round to 3 digits)

From now on, use these columns only:

['seniority', 'income', 'assets', 'records', 'job', 'home']

Apply one-hot-encoding using DictVectorizer and train the logistic regression with these parameters:

LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)

In [18]:
features = ['seniority', 'income', 'assets', 'records', 'job', 'home']

In [19]:
from sklearn.feature_extraction import DictVectorizer

In [20]:
dv = DictVectorizer()

In [21]:
train_dict=df_train[features].to_dict(orient = "records")
train_dict[0]

{'seniority': 14,
 'income': 120,
 'assets': 6500,
 'records': 'no',
 'job': 'fixed',
 'home': 'owner'}

In [22]:
x_train = dv.fit_transform(train_dict)

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000, solver='liblinear')

In [25]:
val_dict = df_val[features].to_dict(orient="records")
x_val = dv.transform(val_dict)

y_pred = model.predict_proba(x_val)[:,1]


In [26]:
roc_auc_score(y_val,y_pred)

0.7898757811309252